# FuseMap Tutorial I: Integrating Spatial Transcriptomics Data Across Image-based Technologies

In this tutorial, we'll demonstrate how to use FuseMap to integrate spatial transcriptomics data from different imaging-based technologies (e.g., MERFISH and STARmap). We'll walk through the process step by step, explaining each component in detail.


## 1. Data preparation

We use spatial mouse brain atlases acquired using MERFISH and STARmap as example datasets. The data can be downloaded from:

- MERFISH data: Zhang et al. ([Nature paper](https://www.nature.com/articles/s41586-023-06808-9#data-availability))
- STARmap data: Shi et al. ([Nature paper](https://www.nature.com/articles/s41586-023-06569-5#data-availability))

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import scanpy as sc
from easydict import EasyDict as edict
from fusemap import seed_all, spatial_integrate, ModelType, setup_logging
import logging
import pandas as pd
seed_all(0)

We recommend users to use the following default hyperparameters, but users can adjust according to their requirements.

## 2. Data preprocessing

In [3]:
# set paths to data
data_dir_list = [
    '/Users/mingzeyuan/Workspace/FuseMap/data/merfish.h5ad',
    '/Users/mingzeyuan/Workspace/FuseMap/data/starmap.h5ad'
]
output_dir = '/Users/mingzeyuan/Workspace/FuseMap/output/integrate_merfish_starmap'
os.makedirs(output_dir, exist_ok=True)

Here users can specify `keep_celltype` and `keep_tissueregion` in arguments to filter to only keep specific cell types or tissue regions in the output results. # Empty string "" means keep all cell types or tissue regions.

In [4]:
args = edict(dict(output_save_dir=output_dir, 
                  keep_celltype="", 
                  keep_tissueregion="", 
                  use_llm_gene_embedding="false", 
                  pretrain_model_path=""))

In [ ]:
setup_logging(args.output_save_dir)

arg_dict = vars(args)
dict_pd = {}
for keys in arg_dict.keys():
    dict_pd[keys] = [arg_dict[keys]]
pd.DataFrame(dict_pd).to_csv(args.output_save_dir  + "config.csv", index=False)
logging.info("\n\n\033[95mArguments:\033[0m \n%s\n\n", vars(args))
logging.info("\n\n\033[95mArguments:\033[0m \n%s\n\n", vars(ModelType))

FuseMap requires both gene expression data and spatial coordinates for each cell/spot as input. The spatial coordinates should be specified in the AnnData object's observation (obs) dataframe with columns "x" and "y".

For spatial transcriptomics data, the coordinates may be stored in different formats:
- As "row"/"col" columns in `data.obs`
- As spatial coordinates in `data.obsm["spatial"]` 
- Directly as "x"/"y" columns in `data.obs`

The code below handles these different formats and standardizes them to "x"/"y" columns in `data.obs`. This standardization is important for FuseMap to properly integrate the spatial information across datasets.

In [ ]:
X_input = []
for ind, data_dir in enumerate(data_dir_list):
    print(f"Loading {data_dir}")
    data = sc.read_h5ad(data_dir)    
    # Handle spatial coordinates
    if "x" not in data.obs.columns:
        if "col" in data.obs.columns and "row" in X.obs.columns:
            data.obs["x"] = data.obs["col"]
            data.obs["y"] = data.obs["row"]
        elif "spatial" in data.obsm.keys():
            data.obs["x"] = data.obsm["spatial"][:,0]
            data.obs["y"] = data.obsm["spatial"][:,1]
        else:
            raise ValueError(f"Please provide spatial coordinates in the obs['x'] and obs['y'] columns for {data_dir}")
    
    # Add metadata
    data.obs['name'] = f'section{ind}'
    data.obs['file_name'] = os.path.basename(data_dir)
    print(f"Loaded {data.shape[0]} cells with {data.shape[1]} genes from {data.obs['file_name'].iloc[0]}")
    X_input.append(data)
    
# Set parameters for integration
kneighbor = ["delaunay"] * len(X_input)
input_identity = ["ST"] * len(X_input)
print(f"Loaded {len(X_input)} datasets")

## 3. Data integration

In [ ]:
spatial_integrate(X_input, args, kneighbor, input_identity)

## 4. Visualization

### read single-cell embedding

In [ ]:
ad_embed=sc.read_h5ad(os.path.join(output_dir, 'ad_celltype_embedding.h5ad'))
sc.pp.neighbors(ad_embed, n_neighbors=50,use_rep='X')
sc.tl.umap(ad_embed)
ax = sc.pl.umap(ad_embed,color='batch',size=1, show=False)
ax.set_title('Single-cell embedding, colored by sample ID')

### read spatial embedding

In [ ]:
ad_embed=sc.read_h5ad(os.path.join(output_dir, 'ad_tissueregion_embedding.h5ad'))
sc.pp.neighbors(ad_embed, n_neighbors=50,use_rep='X')
sc.tl.umap(ad_embed)
ax = sc.pl.umap(ad_embed,color='batch',size=1, show=False)
ax.set_title('Spatial embedding, colored by sample ID')

### read gene embedding

In [ ]:
ad_embed=sc.read_h5ad(os.path.join(output_dir, 'ad_gene_embedding.h5ad'))
sc.pp.neighbors(ad_embed, n_neighbors=50,use_rep='X')
sc.tl.umap(ad_embed)
ax = sc.pl.umap(ad_embed,color='type', size=100,show=False)
ax.set_title('Gene embedding, colored by sample ID')